# 04_신경망 학습

### 4-1. 데이터에서 학습한다.      
   1) 학습이란? : 훈련 데이터로부터 가중치 매개변수의 최적값을 자동으로 획득하는 것.     
   2) 손실 함수 : 신경망이 학습할 수 있도록 하는 지표     
   3) 학습의 목표 : 손실 함수의 결과값을 가장 작게 만드는 가중치 매개변수를 찾는 것    
      * 그 중의 한 방법으로 '함수의 기울기를 활용하는 경사법'      

#### 4-1-1. 

1) 신경망의 특징    
  - 데이터를 보고 학습할 수 있다.(매개변수가 자동으로 결정된다.)      
  - 실제 신경망에서는 수천~수만개의 매개변수가 있다.     
  
2) 데이터 주도 학습    
  - 기계학습에서는 규칙을 찾기 보다는 '데이터에서 기반하여' 문제를 풀어나간다. 
  - 이미지의 '특징'을 추출하고, 특징의 패턴을 기계학습 기술로 학습한다.      
  - 이미지의 특징은 주로 벡터로 기술하며, SIFT, SURF, HOG등의 특징을 활용한다.    
  - 이런 특징을 이용하여 이미지를 벡터로 변환하고, 변환된 벡터를 지도학습 방식의 대표적인 방법인 SVM, KNN 등으로 학습한다.      
  ---
  - 단, __특징을 찾는 방법은 여전히 사람이 설계하므로__ 적절한 특징을 찾고 적용해야 좋은 결과를 얻는다.    
  * 딥러닝은 종단간 기계학습(처음부터 끝까지 사람의 개입 없다)이라고도 한다.     

3) 훈련 데이터와 시험 데이터

- 기계학습 문제는 일반적으로 훈련 데이터와 시험 데이터로 나눠 학습과 실험을 수행하는 것이 일반적이다.    
- 훈련 데이터로 최적의 매개변수를 찾는다.      
- 시험 데이터로 훈련한 모델의 실력을 평가한다.   
> __훈련과 시험 과정을 거쳐 '범용적으로' 사용할 수 있는 모델__ 을 찾는다.      

- 훈련 데이터에 오버피팅(과적합) 되지 않도록 한다.

4) __손실 함수__    

- 신경망 학습(최적의 매개변수를 찾는 여정)에서 활용하는 평가 지표.    
- 일반적으로 평균 제곱 오차와 교차 엔트로피 교차를 사용한다.  

    4-1) 평균 제곱 오차   

In [2]:
import numpy as np

#평균 제곱 오차
def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)

#정답이 2일 때
t=[0,0,1,0,0,0,0,0,0,0]

#(2일 확률이 가장 높다고 예측했을 때)
y=[0.1,0.05,0.6,0.0,0.05,0.1,0.0,0.1,0.0,0.0]
true_value = mean_squared_error(np.array(y), np.array(t))

#(7일 확률이 가장 높다고 예측했을 때)
y=[0.1,0.05,0.1,0.0,0.05,0.1,0.0,0.6,0.0,0.0]
false_value = mean_squared_error(np.array(y), np.array(t))

print("true : ", true_value)
print("false : ", false_value)

true :  0.09750000000000003
false :  0.5975


   * 제대로 예측했을 때와 잘못 예측했을 때의 오차 확률이 크게 차이남을 알 수 있다.    

    4-2) 교차 엔트로피 오차

   - 밑이 e인 자연로그    
   - 정답일 때의 추정(t가 1일때 y)의 자연로그를 계산하는 식

In [3]:
#아주 작은 값인 delta를 구해서 log윗부분 이 0이 되지 않도록

def cross_entropy(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y+delta))

In [5]:
#정답이 2일 때
t=[0,0,1,0,0,0,0,0,0,0]

#(2일 확률이 가장 높다고 예측했을 때)
y=[0.1,0.05,0.6,0.0,0.05,0.1,0.0,0.1,0.0,0.0]
true = cross_entropy(np.array(y), np.array(t))

#(7일 확률이 가장 높다고 예측했을 때)
y=[0.1,0.05,0.1,0.0,0.05,0.1,0.0,0.6,0.0,0.0]
false = cross_entropy(np.array(y), np.array(t))

print("true : ", true_value)
print("false : ", false_value)

true :  0.09750000000000003
false :  0.5975


   4-3) 미니배치 학습     
   - 지금까지 본 함수들은 각 데이터 1개에 대한 손실 함수였다.    
   - 그럼 이제 '전체 데이터에 대한 손실'을 생각한다면?    
   
   > (-0.5) * 시그마(0부터 끝까지)(데이터 1개에 대한 교차 엔트로피 함수)    
   ---
   - 하지만 이렇게 모든 데이터에 대한 손실을 계산하려 한다면, 비용이 많이 든다.   
   - 데이터 일부를 추려 전체의 '근사치' 로 이용한다.    
   - 이러한 학습 방법을 '미니배치' 라고 한다.

In [ ]:
import sys, os
import numpy as np
from datasets.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = \load_mnist(normalize=True, one_hot_label=True)

#무작위로 10장만 빼내기
train_size = x_train.shape[0]

batch_size = 10

#random.choice(총 넘버 수, 뽑을 사이즈)
batch_mask = np.random.choice(train_size, batch_size)

x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

#### 데이터 하나당 교차 엔트로피를 구하는 경우

In [ ]:
#배치용 교차 엔트로피 오차 구하기 함수
def cross_entropy_batch_each(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.num(t * np.log(y+1e-7))/batch_size

#### 정답 클래스의 수가 2 이상일 때의 교차 엔트로피 오차를 구하는 경우

In [1]:
def cross_entropy_batch(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y. reshape(1, y.size)
        
    batch_size=y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t]+ 1e-7))/batch_size

#### 왜 손실함수가 필요한가?

- 왜 정확도를 놔두고 손실함수에 주목하는가?
- 신경망 학습에서는 손실함수를 최소화 하는 값을 찾기 위해 매개변수의 미분을 계산하고, 미분을 단서로 매개변수 값을 갱신한다.   
- 하지만, 정확도를 지표로 할 경우 매개변수 미분이 대부분의 장소에서 0이 되므로 개선점을 찾기 어렵다.    
Q. 왜 정확도의 경우 매개변수 미분이 대부분의 장소에서 0이 되는걸까?

#### 수치 미분      

- 미분 : '특정 순간'의 변화량   
